## Import libraries

In [1]:
# import libraries
import gspread
import MySQLCredentials as mc
import mysql.connector
from oauth2client.service_account import ServiceAccountCredentials

## Initialize variables

In [2]:
# initialize variables for gspread
scope = ['https://spreadsheets.google.com/feeds',
         'https://www.googleapis.com/auth/drive']
creds = ServiceAccountCredentials.from_json_keyfile_name('GoogleSheetsToMySQL.json', scope)
client = gspread.authorize(creds)

In [3]:
# define method to pull data from spreadsheet
def GetSpreadsheetData(sheetName, worksheetIndex):
    sheet = client.open(sheetName).get_worksheet(worksheetIndex)
    return sheet.get_all_values()[1:]

def PreserveNULLValues(listName):
    print('Preserving NULL values...')
    for x in range(len(listName)):
        for y in range(len(listName[x])):
            if listName[x][y] == '':
                listName[x][y] = None
    print('NULL values preserved.')

## Define methods

In [19]:
# define method to write list of data to MySQL table
def WriteToMySQLTable(sql_data, tableName):
    try:
        connection = mysql.connector.connect(
                user       = mc.user,
                password   = mc.password,
                host       = mc.host,
                database   = mc.database
            )
        sql_drop = " DROP TABLE IF EXISTS {} ".format(tableName)
        sql_create_table = """CREATE TABLE {}
                ( FK INT(11),
                  level INT(11),
                  FOREIGN KEY (FK) REFERENCES table1(PK)
                )""".format(tableName)
 
        sql_insert_statement = """INSERT INTO {}
                ( FK,
                  level
                   )
        VALUES ( %s,%s )""".format(tableName)
        cursor = connection.cursor()
        #cursor.execute(sql_drop)
        print('Table {} has been dropped'.format(tableName))
        cursor.execute(sql_create_table)
        print('Table {} has been created'.format(tableName))
        for i in sql_data:
            cursor.execute(sql_insert_statement, i)
        connection.commit()
        print("Table {} successfully updated.".format(tableName))
    except mysql.connector.Error as error :
        connection.rollback()
        print("Error: {}. Table {} not updated!".format(error, tableName))
    finally:
        cursor.execute('SELECT COUNT(*) FROM {}'.format(tableName))
        rowCount = cursor.fetchone()[0]
        print(tableName, 'row count:', rowCount)
        if connection.is_connected():
            cursor.close()
            connection.close()
            print("MySQL connection is closed.")

In [ ]:
# update data in MySQL table
def UpdateMySQLTable(sql_data, tableName):
# we are using a try/except block (also called a try/catch block in other languages) which is good for error handling. It will "try" to execute anything in the "try" block, and if there is an error, it will report the error in the "except" block. Regardless of any errors, the "finally" block will always be executed.
    try:
# Here we include the connection credentials for MySQL. We create a connection object that we pass the credentials to, and notice that we specify the database (which from the mysqlcreds.py file tells us that we will be using the qa_db database so we won't need to include that in any code when executing any MySQL statements
        connection = mysql.connector.connect(
                user       = mc.user,
                password   = mc.password,
                host       = mc.host,
                database   = mc.database
            )
 
        sql_insert_statement = """INSERT INTO {}
                ( Column1,
                  Column2,
                  Column3 )
        VALUES ( %s,%s,%s )""".format(tableName)
        cursor = connection.cursor()
        for i in sql_data:
            cursor.execute(sql_insert_statement, i)
        connection.commit()
        print("Table {} successfully updated.".format(tableName))
    except mysql.connector.Error as error :
        connection.rollback()
        print("Error: {}. Table {} not updated!".format(error, tableName))
    finally:
        cursor.execute('SELECT COUNT(*) FROM {}'.format(tableName))
        rowCount = cursor.fetchone()[0]
        print(tableName, 'row count:', rowCount)
        if connection.is_connected():
            cursor.close()
            connection.close()
            print("MySQL connection is closed.")

## Execute Method(s)

In [20]:
PreserveNULLValues(data)
WriteToMySQLTable(data, 'table2')

Preserving NULL values...
NULL values preserved.
Table table2 has been dropped
Table table2 has been created
Table table2 successfully updated.
table2 row count: 10
MySQL connection is closed.


In [10]:
data = GetSpreadsheetData('GoogleSheetData', 1)
PreserveNULLValues(data)
UpdateMySQLTable(data, 'MyData')

Preserving NULL values...
NULL values preserved.
Table MyData successfully updated.
MyData row count: 20
MySQL connection is closed.
